In [1]:
import sys
import os
import re
import pandas as pd
from collections import defaultdict

In [ ]:
###cell for file directories
def exists(direc, file_name):
    exists_ornah = os.path.isdir(direc)
    while exists_ornah == False:
        direc = input("Directory does not exist.. please enter file directory, i.e '/home/name/Dektop/BDD2204 for the file -- " + file_name + ': \n')
        direc = os.path.normpath(direc)
        exists_ornah = os.path.isdir(direc)
    print('Current directory: ' + os.getcwd() + '\n')
    return(direc)

parent_dir = input("Enter file director, i.e '/home/name/Dektop/BDD2204\n")
parent_dir = exists(parent_dir, 'BDD2204, general folder')

#### JIRA BDD-2204


1) Multi-sample file 451546  
`/home/cthomas/Desktop/cris/projects/BDD2204/files/Standford_batch14_7May2014_Raw.451846.txt`  

2) Need to split first using mapping file:  
`/home/cthomas/Desktop/cris/projects/BDD2204/files/cthomas_849_3007.txt`


In [2]:
#uarray_sdy = "/home/cris/Desktop/cris/projects/BDD2204/files/microarray_file_to_split.txt"
uarray_sdy = parent_dir + 'files/microarrayfile_to_split.txt'
uarray_sdy = exists(uarray_sy, 'microarrayfile_to_split.txt: microarray data that needs to be split?')

#exp_acc="EXP13635"
exp_acc = input('Pleas enter the Experiment account number: \n')

## 1. Get the mapping user def id to EXP SAMPLE ACC NUM
#mappingfile = "/home/cris/Desktop/cris/projects/BDD2204/files/cthomas_849_3007.txt"
mappingfile = parent_dir + 'files/cthomas_849_3007.txt'
mappingfile = exists(mappingfile, 'mapping user def ID to experiment account number')


df = pd.read_table(mappingfile)
df[df.EXP_ACC_NUM == exp_acc].head()

,STUDY_ACC_NUM,USER_DEF_BIOSAMPLE_ID,BIO_SAM_ACC_NUM,EXP_ACC_NUM,EXP_SAMPLE_USER_DEF_ID,EXP_SAMPLE_ACC_NUM,FILE_NAME,EXP_SAMPLE_NAME
7182,SDY311,STANSAMS_15065011020,BS715498,EXP13635,STANSAMS_150650110,ES776588,Standford_batch14_7May2014_Raw.451846.txt,5920666014_H
7183,SDY311,STANSAMS_15087011020,BS715497,EXP13635,STANSAMS_150870110,ES776587,Standford_batch14_7May2014_Raw.451846.txt,5920666011_I
7184,SDY311,STANSAMS_15025011020,BS715496,EXP13635,STANSAMS_150250110,ES776586,Standford_batch14_7May2014_Raw.451846.txt,5920666013_G
7185,SDY311,STANSAMS_15002011020,BS715495,EXP13635,STANSAMS_150020110,ES776585,Standford_batch14_7May2014_Raw.451846.txt,5920666013_C
7186,SDY311,STANSAMS_15047011020,BS715494,EXP13635,STANSAMS_150470110,ES776584,Standford_batch14_7May2014_Raw.451846.txt,5920834001_A


In [7]:
esID = {}
# esID = defaultdict(dict)
#study_acc = "SDY311"
study_acc = input('Enter the Study Account Number: \n')
# 1/ Get mapping user-definedID - EXP_SAMPLE_ACC_NUM from mapping file
with open(mappingfile, "r") as mapf:
    fl = mapf.readline()
    for lines in mapf:
        if re.search(study_acc, lines):
            #lines = lines.strip()
            elements = lines.strip().split("\t")
            if elements[3] == exp_acc and elements[7].strip():
                esID[elements[7]] = elements[5]
                
                #esID[elements[0]][elements[7]] = elements[5]
                
                
# 2/ Get directory path and create output directory
output_path_file = os.path.split(os.path.realpath(uarray_sdy))[0]
sdy_dir = "/".join([output_path_file, study_acc, 'b'])
os.makedirs(sdy_dir, exist_ok=True)

# 3/ Create new files and get the data to populate them.
user_def_ids = {}
sdypvals = defaultdict(dict)
sdynbeads = defaultdict(dict)
sdysignal = defaultdict(dict)
sdyprobes = {} 
sdyfiles = {}
idx_signal = []
idx_pval = []
idx_nbeads = []
userIDs=[]

with open(uarray_sdy, "r") as uarray:
    # a. Get the user defined experiment sample ID:
    hdg = uarray.readline()
    hdg = hdg.strip()
    hdgs = hdg.split("\t")
    for k in hdgs: 
        if re.search("\d", k):
            userdefID = k.split(r".")[0]
            userIDs.append(userdefID)
            user_def_ids[k]=userdefID
    print("processing...")
    noES = []
    # b. Create the ouput files -- format will be SDY_ACC_EXP_ACC_EXP_SAM_ACC_NUM_microarray_result.txt
    for userdef in set(userIDs):
        if userdef not in esID:
            noES.append(userdef)
        else:
            new_filename = "_".join([study_acc, esID[userdef], "gene_expression.txt"])
            path_file = "/".join([sdy_dir, new_filename])
            sdyfiles[userdef]= path_file

            with open(path_file, "w") as out:
                out.write("PROBE_ID\tSYMBOL\tSIGNAL\tPVALUE\tNBEADS\n")
            
    print("files created")
    print("getting data...")
    
    # c. Get the data for each expsample
    for lignes in uarray:
        lignes = lignes.strip()
        values = lignes.split("\t")
        if values[1].strip():
            sdyprobes[values[0]]=values[1]
        else:
            sdyprobes[values[0]]="NA"
            
        for m in range(2,len(values)) :
            # SIGNAL
            if hdgs[m].endswith("Signal"):
                sdysignal[ user_def_ids [ hdgs[m] ] ]   [values[0]] = values[m]
            # PVALUE
            elif hdgs[m].endswith("val"):
                sdypvals[user_def_ids[hdgs[m]]][values[0]] = values[m]
            # NBEADS
            elif hdgs[m].endswith("NBEADS"):
                sdynbeads[user_def_ids[hdgs[m]]][values[0]] = values[m]

print("populating files...")
# 4/ Populate the files: 

for exp_samples in sdyfiles:
    with open(sdyfiles[exp_samples], "a") as singlesplefile:
        for probes in sdyprobes:
            singlesplefile.write("\t".join([probes, sdyprobes[probes],sdysignal[exp_samples][probes],
                                        sdypvals[exp_samples][probes],
                                        sdynbeads[exp_samples][probes]]) + "\n")

new_noes = "_".join([study_acc, "noES_ACC_gene_expression.txt"])
noesfile = "/".join([sdy_dir, new_noes])
noeslines = []
for probes in sdyprobes:
    tmpp = []
    for eachES in noES:
        tmpp.append("\t".join([sdysignal[eachES][probes], sdypvals[eachES][probes], sdynbeads[eachES][probes]]))
    tmp2 = "\t".join(tmpp)
    noeslines.append("\t".join([probes, sdyprobes[probes], tmp2]))
    
with open(noesfile, "w") as noe:
    noe.write("PROBE_ID\tSYMBOL\t")
    addl = []
    for eachnoes in noES:
        addl.append("_".join([eachnoes, "SIGNAL"]))
        addl.append("_".join([eachnoes, "PVALUE"]))
        addl.append("_".join([eachnoes, "NBEADS"]))
    noe.write("\t".join(addl) + "\n")
    noe.write("\n".join(noeslines) + "\n")
            
print("done!")

processing...
files created
getting data...
populating files...
done!



3) Upload to IMMPORT. Upload ticket is ..   cthomas_20160307_12138   
4) Run Query 699 with upload ticket.  
5) old file info id is 451546   
6) prepare BCD script for update of the database   

**Uploaded files**  
Result of Query 699  
`/home/cthomas/Desktop/cris/projects/BDD2204/files/cthomas_699_3019.txt`

In [3]:
#fnms = "/home/cthomas/Desktop/cris/projects/BDD2204/files/cthomas_699_3019.txt"
fnms = parent_dir + 'files/cthomas_699.3019.txt'
fnms = exists(fnms, 'Enter FFNS files: \n')
df = pd.read_table(fnms)

df.head()  


,FILE_INFO_ID,UPLD_TCT_NUM,FILE_DETAIL,FILE_LOCATION,FILE_NAME,FILE_PURPOSE,EXP_ACC_NUM,PROCESSING_ID,PROJECT_ID,DATE_CREATED,DATE_LAST_UPDATED,LAST_UPDATED_BY,RAW_FILE,CREATED_BY,FILESIZE_BYTES,ORIGINAL_PROJECT_ID,ORIGINAL_FILE_NAME
0,597763,cthomas_20160307_12138,Custom,NaN,SDY311_ES776571_gene_expression.597763.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:04.0,2016-03-07 16:40:04.0,cthomas,oracle.sql.BLOB@46803e93,cthomas,1951914,2883,SDY311_ES776571_gene_expression.txt
1,597774,cthomas_20160307_12138,Custom,NaN,SDY311_ES776572_gene_expression.597774.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:05.0,2016-03-07 16:40:05.0,cthomas,oracle.sql.BLOB@847eb1c,cthomas,1958517,2883,SDY311_ES776572_gene_expression.txt
2,597777,cthomas_20160307_12138,Custom,NaN,SDY311_ES776577_gene_expression.597777.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:05.0,2016-03-07 16:40:05.0,cthomas,oracle.sql.BLOB@36477c8,cthomas,1952822,2883,SDY311_ES776577_gene_expression.txt
3,597780,cthomas_20160307_12138,Custom,NaN,SDY311_ES776601_gene_expression.597780.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:05.0,2016-03-07 16:40:05.0,cthomas,oracle.sql.BLOB@71fb3864,cthomas,1951654,2883,SDY311_ES776601_gene_expression.txt
4,597790,cthomas_20160307_12138,Custom,NaN,SDY311_ES776637_gene_expression.597790.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:06.0,2016-03-07 16:40:06.0,cthomas,oracle.sql.BLOB@3648f46e,cthomas,1949622,2883,SDY311_ES776637_gene_expression.txt


In [4]:
## Check uniqueness of items under FILE_NAME
if len(df.FILE_NAME)!=len(set(df.FILE_NAME)):
    print("there are duplicates in list")

*__ for filenames containing EXP_SAM_ACC_NUM :__*

In [5]:
## Extract ES number in Query 699 outputs:
def get_exp_sam_acc_num(nms):
    items = nms.split("_")
    for it in items:
        if re.match("ES\d", it):
            es = it
    return(es)

df["EXP_SAMPLE_ACC_NUM"] = df.FILE_NAME.apply(get_exp_sam_acc_num)
df.head()

,FILE_INFO_ID,UPLD_TCT_NUM,FILE_DETAIL,FILE_LOCATION,FILE_NAME,FILE_PURPOSE,EXP_ACC_NUM,PROCESSING_ID,PROJECT_ID,DATE_CREATED,DATE_LAST_UPDATED,LAST_UPDATED_BY,RAW_FILE,CREATED_BY,FILESIZE_BYTES,ORIGINAL_PROJECT_ID,ORIGINAL_FILE_NAME,EXP_SAMPLE_ACC_NUM
0,597763,cthomas_20160307_12138,Custom,NaN,SDY311_ES776571_gene_expression.597763.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:04.0,2016-03-07 16:40:04.0,cthomas,oracle.sql.BLOB@46803e93,cthomas,1951914,2883,SDY311_ES776571_gene_expression.txt,ES776571
1,597774,cthomas_20160307_12138,Custom,NaN,SDY311_ES776572_gene_expression.597774.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:05.0,2016-03-07 16:40:05.0,cthomas,oracle.sql.BLOB@847eb1c,cthomas,1958517,2883,SDY311_ES776572_gene_expression.txt,ES776572
2,597777,cthomas_20160307_12138,Custom,NaN,SDY311_ES776577_gene_expression.597777.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:05.0,2016-03-07 16:40:05.0,cthomas,oracle.sql.BLOB@36477c8,cthomas,1952822,2883,SDY311_ES776577_gene_expression.txt,ES776577
3,597780,cthomas_20160307_12138,Custom,NaN,SDY311_ES776601_gene_expression.597780.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:05.0,2016-03-07 16:40:05.0,cthomas,oracle.sql.BLOB@71fb3864,cthomas,1951654,2883,SDY311_ES776601_gene_expression.txt,ES776601
4,597790,cthomas_20160307_12138,Custom,NaN,SDY311_ES776637_gene_expression.597790.txt,Archived,NaN,NaN,2883,2016-03-07 16:40:06.0,2016-03-07 16:40:06.0,cthomas,oracle.sql.BLOB@3648f46e,cthomas,1949622,2883,SDY311_ES776637_gene_expression.txt,ES776637


In [6]:
## print the string to use in query 777 containing all EXP_SAM_ACC_NUM to re-link:
tmp=[]
## might need to change EXP_SAM_ACC_NUM and EXP_SAMPLE_ACC_NUM back and forth
for i in set(df.EXP_SAMPLE_ACC_NUM):
    nwstr="'"+i+"'"
    tmp.append(nwstr)
query_text = ",".join(tmp)
print(query_text)

'ES776629','ES776632','ES776642','ES776635','ES776583','ES776633','ES776575','ES776630','ES776638','ES776643','ES776620','ES776597','ES776636','ES776576','ES776605','ES776589','ES776586','ES776621','ES776580','ES776641','ES776600','ES776596','ES776594','ES776623','ES776572','ES776584','ES776608','ES776612','ES776607','ES776606','ES776573','ES776579','ES776591','ES776624','ES776610','ES776595','ES776627','ES776614','ES776602','ES776625','ES776611','ES776603','ES776616','ES776640','ES776592','ES776582','ES776617','ES776585','ES776626','ES776631','ES776590','ES776574','ES776613','ES776637','ES776615','ES776587','ES776577','ES776619','ES776622','ES776634','ES776588','ES776604','ES776639','ES776581','ES776599','ES776598','ES776571','ES776628','ES776618','ES776601','ES776578','ES776593','ES776609'


In [7]:
## For single sample files from splitting multisample:
old_FID = "451846"

tmpp = pd.DataFrame(
    {'EXP_SAMPLE_ACC_NUM': df.EXP_SAMPLE_ACC_NUM, 
     'FILE_INFO_ID': old_FID, 
     'NEW_FILE_INFO_ID': df.FILE_INFO_ID })
tmpp["DATA_FORMAT"] = "Illumina Microarray"
tmpp["RESULT_SCHEMA"] = "OTHER"

tmpp.head()


,EXP_SAMPLE_ACC_NUM,FILE_INFO_ID,NEW_FILE_INFO_ID,DATA_FORMAT,RESULT_SCHEMA
0,ES776571,451846,597763,Illumina Microarray,OTHER
1,ES776572,451846,597774,Illumina Microarray,OTHER
2,ES776577,451846,597777,Illumina Microarray,OTHER
3,ES776601,451846,597780,Illumina Microarray,OTHER
4,ES776637,451846,597790,Illumina Microarray,OTHER


BDC scripts:
1. update file_info file purpose and file detail  
`/home/cthomas/Desktop/cris/projects/BDD2204/bdc/BDC_FILE_INFO_UPD.SDY311_BDD2204.txt`
2. expsample_2_file_info delete old FID + update FID   
`/home/cthomas/Desktop/cris/projects/BDD2204/bdc/BDC_EXP_SAM_2_FILE_INFO_UPD.SDY311_BDD2204.txt`
3. exp sample data format and result schema  
`/home/cthomas/Desktop/cris/projects/BDD2204/bdc/BDC_EXPERIMENTAL_SAMPLE_UPD.SDY311_BDD2204.txt`

In [8]:
output_path_file = os.path.split(os.path.realpath(fnms))[0]
output_path = os.path.split(os.path.realpath(output_path_file))[0]

#NAME OF BDC SCRIPTS
bdc_script_upd_es2fi =  "/".join([output_path, "bdc/BDC_EXP_SAM_2_FILE_INFO_UPD.SDY311_BDD2204.txt"])  
bdc_script_upd_fi =  "/".join([output_path, "bdc/BDC_FILE_INFO_UPD.SDY311_BDD2204.txt"])  
bdc_script_upd_es =  "/".join([output_path, "bdc/BDC_EXPERIMENTAL_SAMPLE_UPD.SDY311_BDD2204.txt"])  

tmp_dir = "/".join([output_path, "tmp"]) 
os.makedirs(tmp_dir, exist_ok=True)

## UPDATE EXP_SAM_2_FILE_INFO
ofile_es2fi = "/".join([output_path, "tmp/bdc_update_exp_sam_2_file.txt"]) 
tmpp.to_csv(ofile_es2fi, sep="\t", header=False, index=False)

with open(bdc_script_upd_es2fi, "w") as bdc_upd_esfi:
    bdc_upd_esfi.write("FORMAT\tUPDATE_LINKAGE\n")
    bdc_upd_esfi.write("TABLE_NAME\tEXP_SAM_2_FILE_INFO\n")
    bdc_upd_esfi.write("KEY_COLUMN\tEXP_SAMPLE_ACC_NUM\n")
    bdc_upd_esfi.write("CHANGING_COLUMN\tFILE_INFO_ID\n")
    bdc_upd_esfi.write("COLUMNS\tDATA_FORMAT~RESULT_SCHEMA\n")
    bdc_upd_esfi.write("DATA STARTED\n")
    with open(ofile_es2fi, "r") as tmp_file1:
        for line in tmp_file1:
            bdc_upd_esfi.write(line)

os.remove(ofile_es2fi)
os.rmdir("/".join([output_path, "tmp"]))

## UPDATE FILE_INFO
with open(bdc_script_upd_fi, "w") as bdc_upd_fi:
    bdc_upd_fi.write("FORMAT\tUPDATE_VAL\n")
    bdc_upd_fi.write("TABLE_NAME\tFILE_INFO\n")
    bdc_upd_fi.write("KEY_COLUMN\tFILE_INFO_ID\n")
    bdc_upd_fi.write("COLUMNS\tFILE_INFO_ID~FILE_DETAIL~FILE_PURPOSE\n")
    bdc_upd_fi.write("DATA STARTED\n")
    for k in set(tmpp.NEW_FILE_INFO_ID):
        bdc_upd_fi.write("\t".join([str(k), "Illumina BeadArray","Gene expression result"]))
        bdc_upd_fi.write("\n")

## UPDATE EXPERIMENTAL_SAMPLE
with open(bdc_script_upd_es, "w") as bdc_upd_es:
    bdc_upd_es.write("FORMAT\tUPDATE_VAL\n")
    bdc_upd_es.write("TABLE_NAME\tEXPERIMENTAL_SAMPLE\n")
    bdc_upd_es.write("KEY_COLUMN\tEXP_SAMPLE_ACC_NUM\n")
    bdc_upd_es.write("COLUMNS\tEXP_SAMPLE_ACC_NUM~EXP_SAMPLE_DATA_FORMAT~EXP_SAMPLE_RESULT_SCHEMA\n")
    bdc_upd_es.write("DATA STARTED\n")
    for l in set(tmpp.EXP_SAMPLE_ACC_NUM):
        bdc_upd_es.write("\t".join([str(l),"Illumina Microarray","OTHER"]))
        bdc_upd_es.write("\n")
